# Run All Models
The `Run-All-Models.ipynb` notebook creates all fluid mixing and energy supply models presented in Trinh+ (2026) JGR Planets, figures 2-4 and S1–S2.

First, we download all necessary libraries and load the AqEquil database.

In [1]:
import numpy as np
import aqequil
import pandas as pd
import matplotlib.pyplot as plt
import pychnosz
import pickle

# load database and AqEquil
ae = aqequil.AqEquil(db='kev', 
                     suppress_redox=["C"],
                     exclude_organics=True,
                     )

Loading a user-supplied thermodynamic database...
data1.kev was not found in the EQ36DA directory but a data0.kev was found in the current working directory. Using it...
data0.kev is now set as the active thermodynamic database.


Next, we define functions to reuse later.

The first function, `createInputs()`, organizes our model parameters into a csv file. This csv file will the be used as an input for the EQ3 software for later speciation. Each csv file is saved in the format "ps_input_" + [number] + ".csv", with each file containing two rows that contain parameters for the cold and hot reservoirs.

The second function, `mixFluids()`, will perform two tasks. First, `mixFluids()` will speciate a pair of cold and hot reservoir fluids. Second, `mixFluids()` will perform a mixing calculation on the cold and hot reservoirs. The saved output will have the form "ps_output_" + [number] + ".pkl".

In [2]:
def createInputs(delta_C, delta_logfO2, delta_pH, T_end, Pres=1e3, filename='ps_recent.csv'):
    '''Create input csv files for subsequent EQ3/6 calculations. Define range of logfO2 (deviation 
    from PPM in log units), pH (deviation from neutral pH), and temperature range.'''
    # define cold and hot fluid parameters
    Ctot_c, Ctot_h = delta_C # molal of total C
    slfO2_c, slfO2_h = delta_logfO2 # deviation from PPM
    spH_c, spH_h = delta_pH # deviation from neutral pH
    T_cold, T_hot = T_end # fluid reservoir temperatures
    
    
    ## obtain relevant equilibrium constants
    # methanogenesis, neutralpH, PPM buffer, and O2 conversion
    data = pychnosz.subcrt(
            species = ['CO2', 'H2O', 'CH4', 'O2'],
            state = ['aq', 'liq', 'aq', 'aq'],
            coeff = [-0.5, -1, 0.5, 1],
            T = T_end,
            P = Pres,
            property="logK",
            messages=False,
            show=False,
            ).out
    
    # data for neutral pH
    data_pH = pychnosz.subcrt(
            species = ['H2O', 'H+', 'OH-'],
            state = ['liq', 'aq', 'aq'],
            coeff = [-1, 1, 1],
            T = T_end,
            P = Pres,
            property="logK",
            messages=False,
            show=False,
            ).out
    pH_neutral = -0.5 * data_pH.logK
    
    # equilibrium between aqueous and gaseous O2
    data_O2 = pychnosz.subcrt(
            species = ['O2', 'O2'],
            state = ['gas', 'aq'],
            coeff = [-1, 1],
            T = T_end,
            P = Pres,
            property="logK",
            messages=False,
            show=False,
            ).out
    
    # PPM buffer
    data_PPM = pychnosz.subcrt(
            species = ['pyrite', 'magnetite', 'pyrrhotite', 'O2'],
            state = ['cr', 'cr', 'cr', 'aq'],
            coeff = [-1.5, -0.5, 3, 1],
            T = T_end,
            P = Pres,
            property="logK",
            messages=False,
            show=False,
            ).out
    logaO2_PPM = data_PPM.logK
    logfO2_PPM = logaO2_PPM - data_O2.logK
    
    # set carbon abundance
    data_C = pychnosz.subcrt(
            species = ['CO2', 'H2O', 'CH4', 'O2'],
            state = ['aq', 'liq', 'aq', 'gas'],
            coeff = [-1, -2, 1, 2],
            T = T_end,
            P = Pres,
            property="logK",
            messages=False,
            show=False,
            ).out

    # define data for input
    cold_logfO2 = logfO2_PPM.iloc[0] + slfO2_c
    hot_logfO2 = logfO2_PPM.iloc[1] + slfO2_h
    cold_pH = pH_neutral.iloc[0] + spH_c
    hot_pH = pH_neutral.iloc[1] + spH_h
    
    cold_Crat = 10**(data_C.logK.iloc[0] - 2*cold_logfO2) # molal ratio of CH4 to CO2
    hot_Crat  = 10**(data_C.logK.iloc[1] - 2*hot_logfO2) # molal ratio of CH4 to CO2
    cold_mCO2 = Ctot_c / (1+cold_Crat) # assume a = m
    hot_mCO2 = Ctot_h / (1+hot_Crat) # assume a = m
    cold_mCH4 = Ctot_c - cold_mCO2
    hot_mCH4 = Ctot_h - hot_mCO2
    input_mCO2 = [cold_mCO2, hot_mCO2]
    input_mCH4 = [cold_mCH4, hot_mCH4]

    reservoirs = ['cold fluid', 'hot fluid']
    input_P = [Pres, Pres]
    input_T = [T_cold, T_hot]
    input_logfO2 = [cold_logfO2, hot_logfO2]
    input_pH = [cold_pH, hot_pH]

    ## write to csv
    data_dict = {'Sample': ['id'] + reservoirs,
         'Pressure': ['bar'] + input_P,
         'Temperature': ['degC'] + input_T,
         'logfO2': ['logfO2'] + input_logfO2,
         'H+': ['pH'] + input_pH,
         'HCO3-': ['Molality'] + input_mCO2,
         'CH4': ['Molality'] + input_mCH4
        }
    df_input = pd.DataFrame(data_dict)
    df_input.to_csv(filename, index=False)

def mixFluids(input_ps, output_ps):
    '''Conduct a parameter sensitivity test for'''

    # speciate in the custom-database-kev file
    ae = aqequil.AqEquil(db="kev")
    speciation = ae.speciate(input_filename=input_ps,
                            )
    speciation.thermo.df_rejected_species # works if a data0 is not used
    speciation.thermo.csv_db = pd.read_csv("wrm_data.csv")

    # EQ3/6 calc for seawater:vent fluid mass ratio <= 1
    Mix = aqequil.Mixing_Fluid(speciation=speciation,
                               sample_name='hot fluid',
                               mass_ratio=1)
    
    r1 = aqequil.Prepare_Reaction(reactants=[Mix],
                                  clear_es_solids_initial=True,
                                  n_steps_print_interval=7,
                                  mineral_suppression_option="All")
    
    speciation1 = aqequil.react(speciation, r1)
    m1 = speciation1.mt("cold fluid")
    
    # EQ3/6 calc for seawater:vent fluid mass ratio >= 1
    Mix = aqequil.Mixing_Fluid(speciation=speciation,
                               sample_name="cold fluid",
                               )
    
    r2 = aqequil.Prepare_Reaction(reactants=[Mix],
                                  n_steps_print_interval=7,
                                  clear_es_solids_initial=True,
                                  mineral_suppression_option="All")
    
    speciation2 = aqequil.react(speciation, r2)
    m2 = speciation2.mt('hot fluid')
    
    # merge EQ3/6 calcs to span wide range of fluid mass ratios
    mixture = aqequil.join_mixes(m1, m2)

    # save mixtures
    with open(output_ps, 'wb') as file: 
        # A new file will be created 
        pickle.dump(mixture, file) 

# Perform and save calculations
Next, we define our model inputs. The most important model here is the "nominal model", which is a model whose values are in the middle of the ranges that we explore for each parameter. Let's call this `modelN` in our code. The other models vary individual model parameters in isolation with respect to the nominal model. The code below labels these models and describes how they differ from the nominal model.

In [3]:
# define speciation inputs
# note: Melwani-Daswani+ (2021) has CO2 molality of 0.33 (CM) and 1.48 (CI)
# model input format: molal_Ctot, delta PPM, delta neutral pH, T_end]

# NOMINAL MODEL
modelN = [[1e-2, 1e-2], [0, 0], [0, 0], [2, 500]] # nominal model

# COLD RESERVOIR SENSITIVITY
model1 = [[1e-2, 1e-2], [0, 0], [-2, 0], [2, 500]] # more acidic
model2 = [[1e-2, 1e-2], [0, 0], [2, 0], [2, 500]] # less acidic

model3 = [[1e-3, 1e-2], [0, 0], [0, 0], [2, 500]] # less C
model4 = [[1e-1, 1e-2], [0, 0], [0, 0], [2, 500]] # more C

model5 = [[1e-2, 1e-2], [0, 0], [0, 0], [10, 500]] # 10 degC
model6 = [[1e-2, 1e-2], [0, 0], [0, 0], [25, 500]] # 25 degC

model7 = [[1e-2, 1e-2], [-3, 0], [0, 0], [2, 500]] # less oxidized
model8 = [[1e-2, 1e-2], [3, 0], [0, 0], [2, 500]] # more oxidized

# HOT RESERVOIR SENSITIVITY
model9 = [[1e-2, 1e-2], [0, 0], [0, -2], [2, 500]] # more acidic
model10 = [[1e-2, 1e-2], [0, 0], [0, 2], [2, 500]] # less acidic

model11 = [[1e-2, 1e-3], [0, 0], [0, 0], [2, 500]] # less C
model12 = [[1e-2, 1e-1], [0, 0], [0, 0], [2, 500]] # more C

model13 = [[1e-2, 1e-2], [0, 0], [0, 0], [2, 300]] # 300 degC
model14 = [[1e-2, 1e-2], [0, 0], [0, 0], [2, 350]] # 350 degC
model15 = [[1e-2, 1e-2], [0, 0], [0, 0], [2, 400]] # 400 degC
model16 = [[1e-2, 1e-2], [0, 0], [0, 0], [2, 450]] # 450 degC

model17 = [[1e-2, 1e-2], [0, -3], [0, 0], [2, 500]] # less oxidized
model18 = [[1e-2, 1e-2], [0, 3], [0, 0], [2, 500]] # more oxidized

models = [modelN, model1, model2, model3, model4, model5, model6, model7, model8,
          model9, model10, model11, model12, model13, model14, model15, model16, model17, model18]

# convert all model inputs into csv input files for later speciation
N = len(models)
for i in range(N):
    mm = models[i]
    s_filename = 'ps_input_' + str(i) + '.csv'
    createInputs(mm[0], mm[1], mm[2], mm[3], filename=s_filename)

Now, we can perform our speciation and mixing calculations! Later, we can use any of the "Make Figure " + [number] notebooks in this folder to read a pkl file to plot the data.

In [4]:
# mix fluids for each speciated file
for i in range(N):
    filename_i = 'ps_input_' + str(i) + '.csv'
    filename_o = 'ps_output_' + str(i) + '.pkl'
    mixFluids(filename_i, filename_o)

Loading a user-supplied thermodynamic database...
data1.kev was not found in the EQ36DA directory but a data0.kev was found in the current working directory. Using it...
data0.kev is now set as the active thermodynamic database.
The input file column 'logfO2' will be used to set sample redox state. If a another column is desired, set it manually using the redox_flag parameter.
Successfully created a data1.kev from data0.kev
Using kev to speciate cold fluid
Using kev to speciate hot fluid
Finished!
Using data0.kev to react cold.fluid
Using data0.kev to react hot.fluid
Using data0.kev to react cold.fluid
Using data0.kev to react hot.fluid
Loading a user-supplied thermodynamic database...
data1.kev was not found in the EQ36DA directory but a data0.kev was found in the current working directory. Using it...
data0.kev is now set as the active thermodynamic database.
The input file column 'logfO2' will be used to set sample redox state. If a another column is desired, set it manually using t